In [1]:
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Attention
from tensorflow.keras import mixed_precision

mixed_precision.set_global_policy("mixed_float16")
tf.config.optimizer.set_jit(True)

vocab_size_input = 10000 + 1
vocab_size_output = 6000 + 1
embedding_dim = 64
units=256
BATCH = 16384
MAXLEN_DOC = 100      # comprimento da sequência do document
MAXLEN_SUM = 30       # comprimento da sequência do summary

2025-07-29 21:54:18.724406: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753836858.738461   48082 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753836858.742705   48082 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753836858.754591   48082 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753836858.754615   48082 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753836858.754616   48082 computation_placer.cc:177] computation placer alr

In [2]:
# ENCODER LAYERS
encoder_inputs = Input(shape=(100,))
enc_emb = Embedding(input_dim=vocab_size_input, output_dim=embedding_dim)(encoder_inputs)

encoder_lstm = LSTM(units, return_sequences=True, return_state=True)
encoder_output, state_h, state_c = encoder_lstm(enc_emb)

# DECODER LAYERS
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(vocab_size_output, embedding_dim)
dec_emb = dec_emb_layer(decoder_inputs)

decoder_lstm = LSTM(units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=[state_h, state_c])

# ATTENTION LAYERS
attention = Attention()
context_vector = attention([decoder_outputs, encoder_output])

decoder_concat_input = Concatenate(axis=-1)([decoder_outputs, context_vector])
decoder_dense = TimeDistributed(Dense(vocab_size_output, activation='softmax'))
final_output = decoder_dense(decoder_concat_input)

model = Model([encoder_inputs, decoder_inputs], final_output)

I0000 00:00:1753836877.465369   48082 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1754 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


In [3]:
model.compile(
    optimizer=tf.keras.optimizers.AdamW(learning_rate=1e-3),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['sparse_categorical_accuracy']
)

In [4]:
# Reconstrua o element_spec manualmente:
element_spec = {
    'document': tf.TensorSpec(shape=(BATCH, MAXLEN_DOC), dtype=tf.int64),
    'summary' : tf.TensorSpec(shape=(BATCH, MAXLEN_SUM),  dtype=tf.int64),
}


In [5]:
ds = tf.data.Dataset.load(
    "vectorized_gigaword_ds",
    element_spec=element_spec
)
ds = ds.unbatch()

In [6]:
print(ds)

<_UnbatchDataset element_spec={'document': TensorSpec(shape=(100,), dtype=tf.int64, name=None), 'summary': TensorSpec(shape=(30,), dtype=tf.int64, name=None)}>


In [7]:
VALIDATION_SPLIT = 0.2
SHUFFLE_BUFFER   = 10000
BATCH_SIZE       = 128

def make_training_example(x):
    enc_in = x['document']                         # (MAXLEN_DOC,)
    dec_in = x['summary'][:-1]                     # (MAXLEN_SUM-1,)
    dec_tr = x['summary'][1:]                      # (MAXLEN_SUM-1,)
    return ( (enc_in, dec_in), dec_tr )

paired = ds.map(make_training_example, num_parallel_calls=tf.data.AUTOTUNE)
shuffled = paired.shuffle(SHUFFLE_BUFFER, reshuffle_each_iteration=False)
total = tf.data.experimental.cardinality(shuffled).numpy()
val_count = int(total * VALIDATION_SPLIT)

val_ds = (
    shuffled
    .take(val_count)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)

train_ds = (
    shuffled
    .skip(val_count)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
    .cache()
)


In [8]:
for batch in ds.take(1):
	print(batch['document'].shape)

(100,)


2025-07-29 21:54:44.656737: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [9]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3,           # para de treinar se não melhorar após 3 épocas
    restore_best_weights=True
)

model.fit(
    train_ds,
    epochs=30,
    validation_data=val_ds,
    callbacks=[early_stop]
)

Epoch 1/30


I0000 00:00:1753836886.946467   50050 service.cc:152] XLA service 0x2e4bfea0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753836886.946511   50050 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
I0000 00:00:1753836886.955619   50050 cuda_dnn.cc:529] Loaded cuDNN version 90701
I0000 00:00:1753836887.012653   50050 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2025-07-29 21:54:54.023634: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.


23754/23860 ━━━━━━━━━━━━━━━━━━━━ 5s 52ms/step - loss: 1.2841 - sparse_categorical_accuracy: 0.8025

2025-07-29 22:15:30.449907: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2025-07-29 22:15:30.449961: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 5195065062337259839
2025-07-29 22:15:30.449965: I tensorflow/core/framework/local_rendezvous.cc:430] Local rendezvous send item cancelled. Key hash: 13385906280107008263
2025-07-29 22:15:30.449970: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 7151457061123746715
2025-07-29 22:15:30.449972: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 10294285862842628588
2025-07-29 22:15:30.449985: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 5010109724675207992
2025-07-29 22:15:30.449989: I tensorflow/core/framework/local_rendezv

23860/23860 ━━━━━━━━━━━━━━━━━━━━ 1320s 55ms/step - loss: 1.2831 - sparse_categorical_accuracy: 0.8026 - val_loss: 0.8333 - val_sparse_categorical_accuracy: 0.8419
Epoch 2/30
23754/23860 ━━━━━━━━━━━━━━━━━━━━ 5s 50ms/step - loss: 0.8091 - sparse_categorical_accuracy: 0.8445

...

23860/23860 ━━━━━━━━━━━━━━━━━━━━ 1255s 53ms/step - loss: 0.6763 - sparse_categorical_accuracy: 0.8593 - val_loss: 0.7206 - val_sparse_categorical_accuracy: 0.8541
Epoch 29/30
23753/23860 ━━━━━━━━━━━━━━━━━━━━ 5s 50ms/step - loss: 0.6759 - sparse_categorical_accuracy: 0.8594

2025-07-30 08:00:45.917604: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 10294285862842628588
2025-07-30 08:00:45.917627: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 5010109724675207992
2025-07-30 08:00:45.917631: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 5195065062337259839
2025-07-30 08:00:45.917648: I tensorflow/core/framework/local_rendezvous.cc:430] Local rendezvous send item cancelled. Key hash: 13385906280107008263
2025-07-30 08:00:45.917657: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 911401776049882864
2025-07-30 08:00:45.917660: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 7151457061123746715
2025-07-30 08:00:45.917663: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv it

23860/23860 ━━━━━━━━━━━━━━━━━━━━ 1251s 52ms/step - loss: 0.6759 - sparse_categorical_accuracy: 0.8594 - val_loss: 0.7205 - val_sparse_categorical_accuracy: 0.8541
Epoch 30/30
23753/23860 ━━━━━━━━━━━━━━━━━━━━ 5s 50ms/step - loss: 0.6755 - sparse_categorical_accuracy: 0.8595

2025-07-30 08:21:36.913869: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 10294285862842628588
2025-07-30 08:21:36.913890: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 5010109724675207992
2025-07-30 08:21:36.913898: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 911401776049882864
2025-07-30 08:21:36.913904: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 1153366086725893780
2025-07-30 08:21:36.913907: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 5195065062337259839
2025-07-30 08:21:36.913915: I tensorflow/core/framework/local_rendezvous.cc:430] Local rendezvous send item cancelled. Key hash: 13385906280107008263
2025-07-30 08:21:36.913919: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv it

23860/23860 ━━━━━━━━━━━━━━━━━━━━ 1251s 52ms/step - loss: 0.6755 - sparse_categorical_accuracy: 0.8595 - val_loss: 0.7204 - val_sparse_categorical_accuracy: 0.8541


In [18]:
# ENCODER MODEL

# 1. Inputs
encoder_inputs = model.input[0]  # equivale a layer 0

# 2. Camadas do encoder
enc_emb_layer = model.get_layer(name='embedding')  # index 2
enc_lstm_layer = model.get_layer(name='lstm')      # index 4

# 3. Reconstrói fluxo
enc_emb = enc_emb_layer(encoder_inputs)
encoder_outputs, state_h_enc, state_c_enc = enc_lstm_layer(enc_emb)

# 4. Cria o model
encoder_model = Model(
    inputs=encoder_inputs,
    outputs=[encoder_outputs, state_h_enc, state_c_enc]
)

# DECODER MODEL


# Parâmetros
units = enc_lstm_layer.units
vocab_size_output = model.get_layer(name='embedding_1').input_dim

# 1. Inputs de inference
decoder_input_token = Input(shape=(1,),            name='decoder_input')     # one token
decoder_state_h_in  = Input(shape=(units,),        name='decoder_state_h')
decoder_state_c_in  = Input(shape=(units,),        name='decoder_state_c')
encoder_outputs_in  = Input(shape=(None, units),    name='encoder_outputs')

# 2. Camadas do decoder (reutilizando pesos)
dec_emb_layer  = model.get_layer(name='embedding_1')     # index 3
dec_lstm_layer = model.get_layer(name='lstm_1')          # index 5
attn_layer     = model.get_layer(name='attention')       # index 6
concat_layer   = model.get_layer(name='concatenate')     # index 7
proj_layer     = model.get_layer(name='time_distributed')# index 8

# 3. Fluxo passo‑a‑passo
dec_emb_inf = dec_emb_layer(decoder_input_token)  
dec_outs, state_h_dec, state_c_dec = dec_lstm_layer(
    dec_emb_inf, initial_state=[decoder_state_h_in, decoder_state_c_in]
)
context_vector = attn_layer([dec_outs, encoder_outputs_in])
dec_concat = concat_layer([dec_outs, context_vector])
dec_logits = proj_layer(dec_concat)

# 4. Modelo de inference do decoder
decoder_model = Model(
    inputs=[decoder_input_token, encoder_outputs_in, decoder_state_h_in, decoder_state_c_in],
    outputs=[dec_logits, state_h_dec, state_c_dec]
)

In [19]:
encoder_model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 100, 64)        │       640,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ [(None, 100, 256),     │       328,704 │
│                                 │ (None, 256), (None,    │               │
│                                 │ 256)]                  │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 968,768 (3.70 MB)

 Trainable params: 968,768 (3.70 MB)

 Non-trainable params: 0 (0.00 B)

In [20]:
decoder_model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ decoder_input       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 1, 64)     │    384,064 │ decoder_input[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_state_h     │ (None, 256)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_state_c     │ (None, 256)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, 1, 256),  │    328,704 │ embedding_1[1][0… │
│                     │ (None, 256),      │            │ decoder_state_h[… │
│                     │ (None, 256)]      │            │ decoder_state_c[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_outputs     │ (None, None, 256) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention           │ (None, 1, 256)    │          0 │ lstm_1[1][0],     │
│ (Attention)         │                   │            │ encoder_outputs[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 1, 512)    │          0 │ lstm_1[1][0],     │
│ (Concatenate)       │                   │            │ attention[1][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed    │ (None, 1, 6001)   │  3,078,513 │ concatenate[1][0] │
│ (TimeDistributed)   │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,791,281 (14.46 MB)

 Trainable params: 3,791,281 (14.46 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
import numpy as np

def decode_sequence(input_seq):
    # 1) codifica o documento
    enc_outs, enc_h, enc_c = encoder_model.predict(input_seq)

    # 2) Início de sequência
    sos = tokenizer_output.word_index['<sos>']
    target_seq = np.array([[sos]])
    decoded = []
    h, c = enc_h, enc_c
    